In [1]:
import pandas as pd
import requests
import json
import datetime as dt

In [2]:
# Save config information.
url = "https://api.the-odds-api.com/v3/odds/?"
key = "ee3dbfa00009d80ce0791275a27f603a"
sport = "baseball_mlb"

In [3]:
# Build partial query URL
ml_query_url = f"{url}apiKey={key}&sport={sport}&region=us"

In [4]:
ml_response = requests.get(ml_query_url).json()

In [5]:
ml_response

{'success': True,
 'data': [{'sport_key': 'baseball_mlb',
   'sport_nice': 'MLB',
   'teams': ['Boston Red Sox', 'New York Yankees'],
   'commence_time': 1596236700,
   'home_team': 'New York Yankees',
   'sites': [{'site_key': 'unibet',
     'site_nice': 'Unibet',
     'last_update': 1596213230,
     'odds': {'h2h': [2.65, 1.48]}},
    {'site_key': 'williamhill_us',
     'site_nice': 'William Hill (US)',
     'last_update': 1596213142,
     'odds': {'h2h': [2.65, 1.56]}},
    {'site_key': 'lowvig',
     'site_nice': 'LowVig.ag',
     'last_update': 1596213125,
     'odds': {'h2h': [2.62, 1.57]}},
    {'site_key': 'draftkings',
     'site_nice': 'DraftKings',
     'last_update': 1596213093,
     'odds': {'h2h': [2.65, 1.53]}},
    {'site_key': 'betrivers',
     'site_nice': 'BetRivers',
     'last_update': 1596213081,
     'odds': {'h2h': [2.65, 1.53]}},
    {'site_key': 'betonlineag',
     'site_nice': 'BetOnline.ag',
     'last_update': 1596213047,
     'odds': {'h2h': [2.62, 1.56, 1

In [6]:
# Build partial query URL
rl_query_url = f"{url}apiKey={key}&sport={sport}&region=us&mkt=spreads"

In [7]:
rl_response = requests.get(rl_query_url).json()

In [8]:
def rl_api(data):
    team = []
    home = []
    date = []
    rl = []
    home_odds = []
    away_odds = []
    odds = []
    
    for game in range(len(data['data'])):
        event = data['data'][game]
        team.append(event['teams'][0])
        team.append(event['teams'][1])
        home.append(1)
        home.append(0)
        time = dt.datetime.fromtimestamp(int(event['commence_time'])).strftime('%Y-%m-%d')
        date.append(time)
        date.append(time)
        home_odds = []
        away_odds = []
        for site in range(len(event['sites'])):
            h_odd = event['sites'][site]['odds']['spreads']['odds'][0]
            a_odd = event['sites'][site]['odds']['spreads']['odds'][1]    
            home_odds.append(h_odd)
            away_odds.append(a_odd)
        if event['sites_count'] == 0:
            odds.append('NaN')
            odds.append('NaN')
            pl.append('NaN')
            pl.append('NaN')
        else:
            h_odds = sum(home_odds)/len(home_odds)
            a_odds = sum(away_odds)/len(away_odds)
            if h_odds > 2:
                h_odds = round((h_odds - 1) * 100)
            elif h_odds == 1:
                h_odds = 'NaN'
            elif h_odds < 2:
                h_odds = round(100 / (1 - h_odds))
            else:
                h_odds = 100
            if a_odds > 2:
                a_odds = round((a_odds - 1) * 100)
            elif a_odds == 1:
                a_odds = 'NaN'
            elif a_odds < 2:
                a_odds = round(100 / (1 - a_odds))
            else:
                a_odds = 100
            odds.append(h_odds)
            odds.append(a_odds)
            rl.append(event['sites'][site]['odds']['spreads']['points'][0])
            rl.append(event['sites'][site]['odds']['spreads']['points'][1])
        
    game_dict = {
        "Team": team,
        "Date": date,
        "Home": home,
        "Run Line": rl,
        "RL": odds
    }
    df = pd.DataFrame(game_dict)
    df = df.round({'Run Line':1,'RL':0})
    return df

In [9]:
def ml_api(data):
    team = []
    home = []
    date = []
    home_odds = []
    away_odds = []
    odds = []
    
    for game in range(len(data['data'])):
        event = data['data'][game]
        team.append(event['teams'][0])
        team.append(event['teams'][1])
        home.append(1)
        home.append(0)
        time = dt.datetime.fromtimestamp(int(event['commence_time'])).strftime('%Y-%m-%d')
        date.append(time)
        date.append(time)
        home_odds = []
        away_odds = []
        for site in range(len(event['sites'])):
            h_odd = event['sites'][site]['odds']['h2h'][0]
            a_odd = event['sites'][site]['odds']['h2h'][1]  
            home_odds.append(h_odd)
            away_odds.append(a_odd)
        if event['sites_count'] == 0:
            odds.append('NaN')
            odds.append('NaN')
        else:
            h_odds = sum(home_odds)/len(home_odds)
            a_odds = sum(away_odds)/len(away_odds)
            if h_odds > 2:
                h_odds = round((h_odds - 1) * 100)
            elif h_odds == 1:
                h_odds = 'NaN'
            elif h_odds < 2:
                h_odds = round(100 / (1 - h_odds))
            else:
                h_odds = 100
            if a_odds > 2:
                a_odds = round((a_odds - 1) * 100)
            elif a_odds == 1:
                a_odds = 'NaN'
            elif a_odds < 2:
                a_odds = round(100 / (1 - a_odds))
            else:
                a_odds = 100
            odds.append(h_odds)
            odds.append(a_odds)
        
    game_dict = {
        "Team": team,
        "Date": date,
        "Home": home,
        "ML": odds
    }
    df = pd.DataFrame(game_dict)
    df = df.round({'ML':0})
    return df

In [10]:
ml_df = ml_api(ml_response)

In [11]:
ml_df

,Team,Date,Home,ML
0,Boston Red Sox,2020-07-31,1,162
1,New York Yankees,2020-07-31,0,-183
2,Atlanta Braves,2020-07-31,1,-132
3,New York Mets,2020-07-31,0,117
4,Cincinnati Reds,2020-07-31,1,-174
5,Detroit Tigers,2020-07-31,0,154
6,Baltimore Orioles,2020-07-31,1,169
7,Tampa Bay Rays,2020-07-31,0,-192
8,Chicago White Sox,2020-07-31,1,-158
9,Kansas City Royals,2020-07-31,0,141


In [12]:
rl_df = rl_api(rl_response)

In [13]:
rl_df

,Team,Date,Home,Run Line,RL
0,Boston Red Sox,2020-07-31,1,1.50,-111
1,New York Yankees,2020-07-31,0,-1.50,-107
2,Atlanta Braves,2020-07-31,1,-1.50,139
3,New York Mets,2020-07-31,0,1.50,-161
4,Cincinnati Reds,2020-07-31,1,-1.5,-112
5,Detroit Tigers,2020-07-31,0,1.5,-105
6,Baltimore Orioles,2020-07-31,1,1.5,106
7,Tampa Bay Rays,2020-07-31,0,-1.5,-124
8,Chicago White Sox,2020-07-31,1,-1.50,-108
9,Kansas City Royals,2020-07-31,0,1.50,-110


In [14]:
ml_df.to_csv('ml_input.csv')

In [15]:
rl_df.to_csv('rl_input.csv')